In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import Autoformer
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)-1]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [5]:
n_inputs = [5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[2,3,4,5,6]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
factor=[2,3,4]
conv_hidden_size=[12,22,32,42,52]
decoder_layers = [1,2,3]

# Hyperparameter Search

In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15 or 10:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 4703606


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 5 | hidden_size: 60 | Loss: MQLoss() | Scaler Type: standard | factor: 2 | decoder_layers: 2
Learning Rate: 3 | decoder_input_size_multiplier: 0.3 | Dropout: 0.3 | n_head: 2 | MovingAvg_window: 15| conv_hidden_size:52
RMSE: 0.0028467213100262464 | QLIKE: 0.027333249667740724 | MAE: 0.0017984156403297639 | MAPE: 0.18038672966811692


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15 or 10:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 33573686


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 84 | hidden_size: 120 | Loss: MAE() | Scaler Type: robust | factor: 2 | decoder_layers: 1
Learning Rate: 2 | decoder_input_size_multiplier: 0.6 | Dropout: 0.2 | n_head: 4 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0033277225505547064 | QLIKE: 0.03944440365864936 | MAE: 0.0021437464355175866 | MAPE: 0.20881335392104441


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if i ==5:
    a2=2
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 47481525


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 50 | Inputs: 15 | hidden_size: 60 | Loss: DistributionLoss() | Scaler Type: standard | factor: 3 | decoder_layers: 1
Learning Rate: 1 | decoder_input_size_multiplier: 0.4 | Dropout: 0.4 | n_head: 3 | MovingAvg_window: 15
RMSE: 0.003006851447221786 | QLIKE: 0.03312854816576531 | MAE: 0.001857494526031283 | MAPE: 0.17341513600432626


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 63165403


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 550 | Inputs: 42 | hidden_size: 180 | Loss: MAE() | Scaler Type: standard | factor: 4 | decoder_layers: 1
Learning Rate: 0 | decoder_input_size_multiplier: 0.6 | Dropout: 0.2 | n_head: 4 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0031114167718332194 | QLIKE: 0.04324932144426737 | MAE: 0.0019893844050930497 | MAPE: 0.19083716206858486


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 80132587


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 15 | hidden_size: 150 | Loss: MSE() | Scaler Type: standard | factor: 2 | decoder_layers: 3
Learning Rate: 0 | decoder_input_size_multiplier: 0.3 | Dropout: 0 | n_head: 4 | MovingAvg_window: 15| conv_hidden_size:12
RMSE: 0.0028266346445326254 | QLIKE: 0.0291242973615545 | MAE: 0.001816219356722326 | MAPE: 0.18342321703352316


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 56597732


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 550 | Inputs: 42 | hidden_size: 150 | Loss: MSE() | Scaler Type: standard | factor: 4 | decoder_layers: 1
Learning Rate: 0 | decoder_input_size_multiplier: 0.3 | Dropout: 0 | n_head: 4 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0029140317241152826 | QLIKE: 0.0314577865817137 | MAE: 0.0018830157680246747 | MAPE: 0.1871712398133047


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 10472761


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 42 | hidden_size: 180 | Loss: MQLoss() | Scaler Type: standard | factor: 2 | decoder_layers: 1
Learning Rate: 2 | decoder_input_size_multiplier: 0.3 | Dropout: 0.3 | n_head: 3 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0037715752585198744 | QLIKE: 0.05346520837738813 | MAE: 0.002563653484989726 | MAPE: 0.25989336585195316


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 14076085


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 42 | hidden_size: 120 | Loss: MQLoss() | Scaler Type: standard | factor: 3 | decoder_layers: 3
Learning Rate: 0 | decoder_input_size_multiplier: 0.6 | Dropout: 0.3 | n_head: 6 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0030393518990663886 | QLIKE: 0.034274578978738694 | MAE: 0.0019238092854621512 | MAPE: 0.1832730857297313


In [ ]:
for z in range (5):
  i=randrange(6)
  h=randrange(5)
  m=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(3)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(3)
  if n_inputs[i]==5:
    a2=0
  if n_inputs[i]==15:
    a2=0
  else:
    a2=a2
  model = Autoformer(h=1, input_size=n_inputs[i],
             decoder_input_size_multiplier=decoder_input_size_multiplier[a1], hidden_size=hidden_size[h],
           dropout=dropout[m], factor=factor[a2], n_head=n_head[a],
           conv_hidden_size=conv_hidden_size[a3],
             decoder_layers=decoder_layers[a5],
             MovingAvg_window=15, loss=losses[m],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {hidden_size[h]} | Loss: {losses[m]} | Scaler Type: {scaler_type[o]} | factor: {factor[a2]} | decoder_layers: {decoder_layers[a5]}")
  print(f"Learning Rate: {n} | decoder_input_size_multiplier: {decoder_input_size_multiplier[a1]} | Dropout: {dropout[m]} | n_head: {n_head[a]} | MovingAvg_window: {15}| conv_hidden_size:{conv_hidden_size[a3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 76707156


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 5 | hidden_size: 150 | Loss: DistributionLoss() | Scaler Type: minmax | factor: 2 | decoder_layers: 2
Learning Rate: 3 | decoder_input_size_multiplier: 0.5 | Dropout: 0.4 | n_head: 2 | MovingAvg_window: 15| conv_hidden_size:22
RMSE: 0.0030188674592545096 | QLIKE: 0.03625652190067799 | MAE: 0.0018455521369970828 | MAPE: 0.18569634350330846


INFO:lightning_fabric.utilities.seed:Seed set to 289990

Epoch: 750 | Inputs: 5 | hidden_size: 120 | Loss: MAE() | Scaler Type: robust | factor: 2 | decoder_layers: 2
Learning Rate: 0 | decoder_input_size_multiplier: 0.4 | Dropout: 0.2 | n_head: 6 | MovingAvg_window: 15| conv_hidden_size:32
RMSE: 0.0026014803693844026 | QLIKE: 0.023928809728980475 | MAE: 0.0015433653548939167 | MAPE: 0.1489220579925518

In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 70936270


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 64113249
RMSE: 0.0026159619758729545 | QLIKE: 0.024138035876253617 | MAE: 0.0015518282598773682 | MAPE: 0.14933247211726608


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 123432993


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 80292897
RMSE: 0.002638165546353872 | QLIKE: 0.024427735670265495 | MAE: 0.0015743939379520462 | MAPE: 0.15183510045932794


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 18066954


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 45175497
RMSE: 0.00262837988890831 | QLIKE: 0.024434129430527057 | MAE: 0.0015758797359756859 | MAPE: 0.15187126054586858


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "Autoformer-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["Autoformer"].values
  else:
    Y_hat=forecasts["Autoformer-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 14276172


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 115818663
RMSE: 0.0026346421327691125 | QLIKE: 0.024392042685290966 | MAE: 0.0015719645391619253 | MAPE: 0.1511529141088067


# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# H=1

In [ ]:
model = Autoformer(h=1, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0038742636604709394 | QLIKE: 0.03130798284496256 | MAE: 0.0023975643290157444 | MAPE: 0.1796906327216747


In [ ]:
if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [ ]:
df2.to_excel("Autoformer forecasts.xlsx")

# H=5

In [7]:
H=5

model = Autoformer(h=H, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Autoformer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004709075667739795 | QLIKE: 0.04569864820047234 | MAE: 0.0028575014925553166 | MAPE: 0.21530123687944494


# H=10

In [8]:
H=10

model = Autoformer(h=H, input_size=5,
             decoder_input_size_multiplier=0.4, hidden_size=120,
           dropout=0.2, factor=2, n_head=6,
           conv_hidden_size=32,
             decoder_layers=2,
             MovingAvg_window=15, loss=losses[1],
             max_steps=750,
           learning_rate=learning_rate[0], num_lr_decays=num_lr_decays[0],
             val_check_steps=500, batch_size=32,
             scaler_type='robust',
              random_seed=289990)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "Autoformer-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["Autoformer"].values
else:
  Y_hat=forecasts["Autoformer-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "Autoformer-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","Autoformer-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("Autoformer forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 289990


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.00570747087072068 | QLIKE: 0.060183553127669286 | MAE: 0.0033522360643235133 | MAPE: 0.24450347549933338
